# Assignment 3

In this assigment, we will work with the *Forest Fire* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/162/forest+fires). Extract the data files into the subdirectory: `../data/fires/` (relative to `./src/`).

## Objective

+ The model objective is to predict the area affected by forest fires given the features set. 
+ The objective of this exercise is to assess your ability to construct and evaluate model pipelines.
+ Please note: the instructions are not meant to be 100% prescriptive, but instead they are a set of minimum requirements. If you find predictive performance gains by applying additional steps, by all means show them. 

## Variable Description

From the description file contained in the archive (`forestfires.names`), we obtain the following variable descriptions:

1. X - x-axis spatial coordinate within the Montesinho park map: 1 to 9
2. Y - y-axis spatial coordinate within the Montesinho park map: 2 to 9
3. month - month of the year: "jan" to "dec" 
4. day - day of the week: "mon" to "sun"
5. FFMC - FFMC index from the FWI system: 18.7 to 96.20
6. DMC - DMC index from the FWI system: 1.1 to 291.3 
7. DC - DC index from the FWI system: 7.9 to 860.6 
8. ISI - ISI index from the FWI system: 0.0 to 56.10
9. temp - temperature in Celsius degrees: 2.2 to 33.30
10. RH - relative humidity in %: 15.0 to 100
11. wind - wind speed in km/h: 0.40 to 9.40 
12. rain - outside rain in mm/m2 : 0.0 to 6.4 
13. area - the burned area of the forest (in ha): 0.00 to 1090.84 









### Specific Tasks

+ Construct four model pipelines, out of combinations of the following components:

    + Preprocessors:

        - A simple processor that only scales numeric variables and recodes categorical variables.
        - A transformation preprocessor that scales numeric variables and applies a non-linear transformation.
    
    + Regressor:

        - A baseline regressor, which could be a [K-nearest neighbours model](https://open.spotify.com/track/4R3AU2pjv8ge2siX1fVbZs?si=b2712f32da0e4358) or a linear model like [Lasso](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html) or [Ridge Regressors](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ridge_regression.html).
        - An advanced regressor of your choice (e.g., Bagging, Boosting, SVR, etc.). TIP: select a tree-based method such that it does not take too long to run SHAP further below. 

+ Evaluate tune and evaluate each of the four model pipelines. 

    - Select a [performance metric](https://scikit-learn.org/stable/modules/linear_model.html) out of the following options: explained variance, max error, root mean squared error (RMSE), mean absolute error (MAE), r-squared.
    - *TIPS*: 
    
        * Out of the suggested metrics above, [some are correlation metrics, but this is a prediction problem](https://www.tmwr.org/performance#performance). Choose wisely (and don't choose the incorrect options.) 

+ Select the best-performing model and explain its predictions.

    - Provide local explanations.
    - Obtain global explanations and recommend a variable selection strategy.

+ Export your model as a pickle file.


You can work on the Jupyter notebook, as this experiment is fairly short (no need to use sacred). 

# Load the data

Place the files in the ../../05_src/data/fires/ directory and load the appropriate file. 

In [3]:
# Load the libraries as required.
import pandas as pd
import os


In [4]:
# Load data
columns = [
    'coord_x', 'coord_y', 'month', 'day', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain', 'area' 
]
fires_dt = (pd.read_csv('C:/Users/huiho/production/05_src/data/fires/forestfires.csv', header = 0, names = columns))
fires_dt.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517 entries, 0 to 516
Data columns (total 13 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   coord_x  517 non-null    int64  
 1   coord_y  517 non-null    int64  
 2   month    517 non-null    object 
 3   day      517 non-null    object 
 4   ffmc     517 non-null    float64
 5   dmc      517 non-null    float64
 6   dc       517 non-null    float64
 7   isi      517 non-null    float64
 8   temp     517 non-null    float64
 9   rh       517 non-null    int64  
 10  wind     517 non-null    float64
 11  rain     517 non-null    float64
 12  area     517 non-null    float64
dtypes: float64(8), int64(3), object(2)
memory usage: 52.6+ KB


# Get X and Y

Create the features data frame and target data.

In [5]:
# Create the features dataframe X
X = fires_dt.drop(columns=['area'])

# Create the target dataframe Y
Y = fires_dt['area']

# Display the first few rows of X and Y
print("Features (X):")
print(X.head())
print("\nTarget (Y):")
print(Y.head())


Features (X):
   coord_x  coord_y month  day  ffmc   dmc     dc  isi  temp  rh  wind  rain
0        7        5   mar  fri  86.2  26.2   94.3  5.1   8.2  51   6.7   0.0
1        7        4   oct  tue  90.6  35.4  669.1  6.7  18.0  33   0.9   0.0
2        7        4   oct  sat  90.6  43.7  686.9  6.7  14.6  33   1.3   0.0
3        8        6   mar  fri  91.7  33.3   77.5  9.0   8.3  97   4.0   0.2
4        8        6   mar  sun  89.3  51.3  102.2  9.6  11.4  99   1.8   0.0

Target (Y):
0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: area, dtype: float64


# Preprocessing

Create two [Column Transformers](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html), called preproc1 and preproc2, with the following guidelines:

- Numerical variables

    * (Preproc 1 and 2) Scaling: use a scaling method of your choice (Standard, Robust, Min-Max). 
    * Preproc 2 only: 
        
        + Choose a transformation for any of your input variables (or several of them). Evaluate if this transformation is convenient.
        + The choice of scaler is up to you.

- Categorical variables: 
    
    * (Preproc 1 and 2) Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) where appropriate.


+ The only difference between preproc1 and preproc2 is the non-linear transformation of the numerical variables.
    


### Preproc 1

Create preproc1 below.

+ Numeric: scaled variables, no other transforms.
+ Categorical: one-hot encoding.

In [6]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# Define the numerical and categorical feature columns
numerical_features = ['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain']
categorical_features = ['month', 'day']

# Create the ColumnTransformer for preproc1
preproc1 = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ]
)

# Display preproc1
print(preproc1)


ColumnTransformer(transformers=[('num', StandardScaler(),
                                 ['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc',
                                  'isi', 'temp', 'rh', 'wind', 'rain']),
                                ('cat', OneHotEncoder(handle_unknown='ignore'),
                                 ['month', 'day'])])


### Preproc 2

Create preproc1 below.

+ Numeric: scaled variables, non-linear transformation to one or more variables.
+ Categorical: one-hot encoding.

In [7]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PowerTransformer, OneHotEncoder

# Create the ColumnTransformer for preproc2
preproc2 = ColumnTransformer(
    transformers=[
        ('num', Pipeline(steps=[
            ('scaler', StandardScaler()),
            ('power', PowerTransformer())
        ]), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ]
)


# Display preproc2
print(preproc2)


ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('scaler', StandardScaler()),
                                                 ('power',
                                                  PowerTransformer())]),
                                 ['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc',
                                  'isi', 'temp', 'rh', 'wind', 'rain']),
                                ('cat', OneHotEncoder(handle_unknown='ignore'),
                                 ['month', 'day'])])


## Model Pipeline


Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `regressor` and assign a regression model to it. 

## Regressor

+ Use a regression model to perform a prediction. 

    - Choose a baseline regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Choose a more advance regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Both model choices are up to you, feel free to experiment.

In [8]:
# Pipeline A = preproc1 + baseline
from sklearn.neighbors import KNeighborsRegressor

pipeline_A = Pipeline(steps=[
    ('preprocessing', preproc1),
    ('regressor', KNeighborsRegressor())
])

print("Pipeline A created successfully.")


Pipeline A created successfully.


In [9]:
# Pipeline B = preproc2 + baseline

pipeline_B = Pipeline(steps=[
    ('preprocessing', preproc2),
    ('regressor', KNeighborsRegressor())
])

print("Pipeline B created successfully.")


Pipeline B created successfully.


In [10]:
# Pipeline C = preproc1 + advanced model
from sklearn.ensemble import RandomForestRegressor

pipeline_C = Pipeline(steps=[
    ('preprocessing', preproc1),
    ('regressor', RandomForestRegressor())
])

print("Pipeline C created successfully.")


Pipeline C created successfully.


In [11]:
# Pipeline D = preproc2 + advanced model

pipeline_D = Pipeline(steps=[
    ('preprocessing', preproc2),
    ('regressor', RandomForestRegressor())
])

print("Pipeline D created successfully.")

    

Pipeline D created successfully.


# Tune Hyperparams

+ Perform GridSearch on each of the four pipelines. 
+ Tune at least one hyperparameter per pipeline.
+ Experiment with at least four value combinations per pipeline.

In [12]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid for Pipeline A
param_grid_A = {
    'regressor__n_neighbors': [3, 5, 7, 9]
}

# Perform GridSearchCV for Pipeline A
grid_search_A = GridSearchCV(pipeline_A, param_grid=param_grid_A, cv=5, scoring='neg_mean_squared_error')
grid_search_A.fit(X, Y)

print("Best parameters for Pipeline A:", grid_search_A.best_params_)
print("Best score for Pipeline A:", grid_search_A.best_score_)



Best parameters for Pipeline A: {'regressor__n_neighbors': 9}
Best score for Pipeline A: -5031.7591945117065


In [13]:
# Define the parameter grid for Pipeline B
param_grid_B = {
    'regressor__n_neighbors': [3, 5, 7, 9]
}

# Perform GridSearchCV for Pipeline B
grid_search_B = GridSearchCV(pipeline_B, param_grid=param_grid_B, cv=5, scoring='neg_mean_squared_error')
grid_search_B.fit(X, Y)

print("Best parameters for Pipeline B:", grid_search_B.best_params_)
print("Best score for Pipeline B:", grid_search_B.best_score_)


Best parameters for Pipeline B: {'regressor__n_neighbors': 9}
Best score for Pipeline B: -5122.853565840271


In [14]:
from sklearn.ensemble import RandomForestRegressor

# Define the parameter grid for Pipeline C
param_grid_C = {
    'regressor__n_estimators': [50, 100, 150, 200]
}

# Perform GridSearchCV for Pipeline C
grid_search_C = GridSearchCV(pipeline_C, param_grid=param_grid_C, cv=5, scoring='neg_mean_squared_error')
grid_search_C.fit(X, Y)

print("Best parameters for Pipeline C:", grid_search_C.best_params_)
print("Best score for Pipeline C:", grid_search_C.best_score_)


Best parameters for Pipeline C: {'regressor__n_estimators': 150}
Best score for Pipeline C: -4785.652386887287


In [15]:
# Define the parameter grid for Pipeline D
param_grid_D = {
    'regressor__n_estimators': [50, 100, 150, 200]
}

# Perform GridSearchCV for Pipeline D
grid_search_D = GridSearchCV(pipeline_D, param_grid=param_grid_D, cv=5, scoring='neg_mean_squared_error')
grid_search_D.fit(X, Y)

print("Best parameters for Pipeline D:", grid_search_D.best_params_)
print("Best score for Pipeline D:", grid_search_D.best_score_)


Best parameters for Pipeline D: {'regressor__n_estimators': 100}
Best score for Pipeline D: -4801.585216317813


# Evaluate

+ Which model has the best performance?

Pipeline C has the best performance

# Export

+ Save the best performing model to a pickle file.

In [16]:
import pickle

# Save the best performing model to a pickle file
with open('best_model.pkl', 'wb') as file:
    pickle.dump(grid_search_C.best_estimator_, file)

print("Best performing model saved to 'best_model.pkl'.")


Best performing model saved to 'best_model.pkl'.


# Explain

+ Use SHAP values to explain the following only for the best-performing model:

    - Select an observation in your test set and explain which are the most important features that explain that observation's specific prediction.

    - In general, across the complete training set, which features are the most and least important.

+ If you were to remove features from the model, which ones would you remove? Why? How would you test that these features are actually enhancing model performance?

In [17]:
import pickle

# Load the best performing model from the pickle file
with open('best_model.pkl', 'rb') as file:
    best_model = pickle.load(file)


In [18]:
# Select an observation from the test set
observation = X.iloc[[0]]  # Using double brackets to keep it as a DataFrame

# Apply the preprocessing step to the entire dataset and then to the observation
preprocessed_data = best_model.named_steps['preprocessing'].fit_transform(X)
observation_transformed = best_model.named_steps['preprocessing'].transform(observation)



In [37]:
import shap
import numpy as np
import pandas as pd

explainer = shap.TreeExplainer(best_model.named_steps['regressor'])
shap_values = explainer.shap_values(preprocessed_data)
observation = X.iloc[[0]]
observation_transformed = best_model.named_steps['preprocessing'].transform(observation)

shap_values_observation = explainer.shap_values(observation_transformed)

feature_names = best_model.named_steps['preprocessing'].get_feature_names_out()

shap_df = pd.DataFrame({
    'Feature': feature_names,
    'SHAP Value': shap_values_observation[0]
})

shap_df['abs_SHAP'] = np.abs(shap_df['SHAP Value'])
shap_df = shap_df.sort_values(by='abs_SHAP', ascending=False)

print(shap_df[['Feature', 'SHAP Value']])



           Feature  SHAP Value
6        num__temp   -5.072925
3         num__dmc   -3.457246
8        num__wind    1.741497
15  cat__month_jul   -1.129832
4          num__dc    0.998520
1     num__coord_y   -0.927036
21  cat__month_sep   -0.856570
7          num__rh   -0.741868
2        num__ffmc    0.613909
24    cat__day_sat   -0.449953
23    cat__day_mon   -0.386893
5         num__isi    0.250094
25    cat__day_sun   -0.243619
27    cat__day_tue   -0.233995
22    cat__day_fri    0.198461
26    cat__day_thu   -0.185919
28    cat__day_wed   -0.111698
13  cat__month_feb   -0.104303
12  cat__month_dec   -0.074080
20  cat__month_oct   -0.073659
18  cat__month_may   -0.072907
17  cat__month_mar    0.066643
0     num__coord_x    0.035983
10  cat__month_apr   -0.015272
11  cat__month_aug   -0.008051
16  cat__month_jun   -0.002705
9        num__rain   -0.001375
14  cat__month_jan    0.000966
19  cat__month_nov    0.000000


*(Answer here.)*

Temperature (num__temp) is the most important feature. SHAP Value: -5.07. Indicates that temperature has a substantial impact on the predicted burned area for this observation.


Most Important Features: Features with the highest mean absolute SHAP values are the most important.
Least Important Features: Features with the lowest mean absolute SHAP values are the least important.

Remove features: cat__month_jan, cat__month_nov... Those one with lowest mean absolute SHAP, which means minimal contribution to the model predictions.

1. Evaluate the performance of current model using cross-validation metrics
2. Remove above features
3. Re-Evaluate the performance of new model using same cross-validation metrics
4. Compare the result with step 1 finding to see if there is an improvement or no changes
5. Use statistical tests to determine if the performance difference between step 1 model and the new model (step 3 model) is statistically significant


## Criteria

The [rubric](./assignment_3_rubric_clean.xlsx) contains the criteria for assessment.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-3`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_3.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Cortez,Paulo and Morais,Anbal. (2008). Forest Fires. UCI Machine Learning Repository. https://doi.org/10.24432/C5D88D.